# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [1]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/refuerzo"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [2]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

Archivos en el directorio: 
['Proyecto_practico.ipynb']


---
### 1.4. Instalar librerías necesarias

In [3]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-8ot46myw
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-8ot46myw
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


In [4]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
import tensorflow as tf
import os

print("TensorFlow versión:", tf.__version__)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("✅ GPU detectada:", gpus)
    tf.config.experimental.set_memory_growth(gpus[0], True)
else:
    print("❌ No se detecta GPU. Ve a Entorno de ejecución > Cambiar tipo de entorno de ejecución > GPU")

tf.config.experimental.enable_tensor_float_32_execution(True)

physical_devices = tf.config.list_physical_devices('GPU')

print("GPU:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(physical_devices))

TensorFlow versión: 2.5.3
❌ No se detecta GPU. Ve a Entorno de ejecución > Cambiar tipo de entorno de ejecución > GPU
GPU: []
Num GPUs: 0


---
### 1.5. Definir directorios de pesos y checkpoints

In [6]:
WEIGHTS_DIR = os.path.join(drive_root, "weights")
CHECKPOINT_DIR = os.path.join(drive_root, "checkpoints")
# Asegurar que existen los directorios
os.makedirs(WEIGHTS_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print("Directorio de pesos:", WEIGHTS_DIR)
print("Directorio de checkpoints:", CHECKPOINT_DIR)
print("Archivos existentes en checkpoints:", os.listdir(CHECKPOINT_DIR))

Directorio de pesos: /content/gdrive/My Drive/refuerzo/weights
Directorio de checkpoints: /content/gdrive/My Drive/refuerzo/checkpoints
Archivos existentes en checkpoints: ['dqn_Model_1_steps.txt', 'dqn_Model_1.h5', 'dqn_Model_2.h5', 'dqn_Model_2_steps.txt']


---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [5]:
from __future__ import division

from PIL import Image
import numpy as np
import gym
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
import os
import glob
from rl.callbacks import Callback
import re
import pickle

#### Configuración base

In [6]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n
print("Las acciones son : ",nb_actions," y son las siguientes:")

Las acciones son :  6  y son las siguientes:


In [7]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal

In [ ]:
# prompt: implementa una red neuronal para trabajar con el spaceinvaders de keras rl
model_name = 'Model_1'
weights_path = os.path.join(CHECKPOINT_DIR, f'dqn_{model_name}.h5')           # pesos
steps_path   = os.path.join(CHECKPOINT_DIR, f'dqn_{model_name}_steps.txt')
final_weights_path = os.path.join(WEIGHTS_DIR, f'dqn_{model_name}.h5')

input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=input_shape))
model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 84, 84, 4)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 32)        8224      
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 512)               1606144   
                                                                 
 dense_1 (Dense)             (None, 6)                 3

2. Implementación de la solución DQN

In [ ]:
#Hiperparámetros
# Hiperparámetros optimizados para Space Invaders DQN
MEMORY_LIMIT = 1000000          # Aumentado para mejor experiencia replay
NB_STEPS = 1200000             # Mínimo recomendado para resultados decentes
NB_STEPS_WARMUP = 20000        # Mantener, es adecuado
GAMMA = 0.99                   # Valor estándar para Atari games
TARGET_MODEL_UPDATE = 10000    # Mantener, es correcto
TRAIN_INTERVAL = 4             # Frecuencia de entrenamiento estándar para DQN
LEARNING_RATE = 0.00025        # Valor estándar y probado para DQN
CALLBACK_INTERVAL = 10000      # Aumentado para mejor monitoreo
CALLBACK_INTERVAL_LOGGER = 10000  # Mantener consistente
FIT_NB_STEPS = 1000000         # Mantener igual que NB_STEPS
LOG_INTERVAL = 10000           # Aumentado para logs más espaciados
MODEL_CHECKPOINT_STEPS = 1000


In [ ]:
memory = SequentialMemory(limit=MEMORY_LIMIT, window_length=WINDOW_LENGTH)

In [ ]:
processor = AtariProcessor()

In [ ]:
policy = BoltzmannQPolicy()

In [ ]:
#implementacion checkpoints

def load_steps(steps_file):
    if os.path.exists(steps_file):
        with open(steps_file, 'r') as f:
            try:
                data=int(f.read().strip())
                print(data)
                return data
            except ValueError:
                pass
    return 0

class SaveLastWeights(Callback):                           # [3]
    def __init__(self, weights_path, steps_path,
                 interval_steps=10_000):
        super().__init__()
        self.weights_path   = weights_path
        self.steps_path     = steps_path
        self.interval_steps = interval_steps
        self.global_step    = load_steps(steps_path)       # contador persistente

    def on_step_end(self, _step, logs=None):
        # _step no es global: lo ignoramos y llevamos nuestro propio contador
        self.global_step += 1
        if self.global_step % self.interval_steps == 0:
            self.model.save_weights(self.weights_path, overwrite=True)
            with open(self.steps_path, 'w') as f:
                f.write(str(self.global_step))
            print(f'💾  Checkpoint guardado | paso global: {self.global_step:,}')

In [ ]:
# Define the agent
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory,
               processor=processor,
               nb_steps_warmup=NB_STEPS_WARMUP,
               gamma=GAMMA,
               target_model_update=TARGET_MODEL_UPDATE,
               train_interval=TRAIN_INTERVAL,
               policy=policy)

dqn.compile(Adam(learning_rate=LEARNING_RATE), metrics=['mae'])

In [ ]:
# En lugar de crear el logger al inicio
# logger = FileLogger('training_log.json', interval=LOG_INTERVAL)

steps_done = load_steps(steps_path)
if os.path.exists(weights_path):
    dqn.load_weights(weights_path)
    print(f'✅  Pesos cargados| pasos completados: {steps_done:,}')
else:
    print('🆕  Entrenamiento nuevo (sin checkpoint previo)')

remaining_steps = max(1, NB_STEPS - steps_done)

902000
✅  Pesos cargados| pasos completados: 902,000


In [ ]:
#entrenar
save_cb = SaveLastWeights(weights_path, steps_path,
                          interval_steps=MODEL_CHECKPOINT_STEPS)

# Entrenar solo los pasos restantes
try:
    dqn.fit(env,
            nb_steps=remaining_steps,
            callbacks=[save_cb],
            log_interval=LOG_INTERVAL,
            verbose=2)
except KeyboardInterrupt:
    print("⏹️  Entrenamiento interrumpido manualmente")

# Guardar pesos finales con nombre del modelo
final_steps = load_steps(steps_path)
dqn.save_weights(final_weights_path, overwrite=True)
with open(steps_path, 'w') as f:
    f.write(str(final_steps))
print(f'Entrenamiento detenido. Pasos totales: {final_steps:,}')


902000
Training for 298000 steps ...


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    615/298000: episode: 1, duration: 9.758s, episode steps: 615, steps per second:  63, episode reward:  5.000, mean reward:  0.008 [ 0.000,  1.000], mean action: 2.481 [0.000, 5.000],  loss: --, mae: --, mean_q: --
💾  Checkpoint guardado | paso global: 903,000
   1371/298000: episode: 2, duration: 4.945s, episode steps: 756, steps per second: 153, episode reward: 14.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 2.439 [0.000, 5.000],  loss: --, mae: --, mean_q: --
💾  Checkpoint guardado | paso global: 904,000
   2154/298000: episode: 3, duration: 5.908s, episode steps: 783, steps per second: 133, episode reward: 11.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.521 [0.000, 5.000],  loss: --, mae: --, mean_q: --
💾  Checkpoint guardado | paso global: 905,000
   3300/298000: episode: 4, duration: 7.513s, episode steps: 1146, steps per second: 153, episode reward: 23.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 2.462 [0.000, 5.000],  loss: --, mae: --, mean

/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  20098/298000: episode: 27, duration: 10.362s, episode steps: 599, steps per second:  58, episode reward: 13.000, mean reward:  0.022 [ 0.000,  1.000], mean action: 2.568 [0.000, 5.000],  loss: 0.043390, mae: 3.382897, mean_q: 4.102926
  20965/298000: episode: 28, duration: 57.552s, episode steps: 867, steps per second:  15, episode reward: 15.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.495 [0.000, 5.000],  loss: 0.030039, mae: 3.354582, mean_q: 4.070060
💾  Checkpoint guardado | paso global: 923,000
  21438/298000: episode: 29, duration: 28.703s, episode steps: 473, steps per second:  16, episode reward:  7.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.531 [0.000, 5.000],  loss: 0.034422, mae: 3.350110, mean_q: 4.062657
💾  Checkpoint guardado | paso global: 924,000
  22083/298000: episode: 30, duration: 39.412s, episode steps: 645, steps per second:  16, episode reward: 12.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 2.561 [0.000, 5.000],  loss: 0.

In [ ]:
weights_filename = os.path.join(WEIGHTS_DIR, f'dqn_{model_name}.h5')

if not os.path.exists(weights_filename):
    raise FileNotFoundError(f'❌  No se encontró {weights_filename}')
dqn.load_weights(weights_filename)
print(f'✅  Pesos cargados desde {weights_filename}')
history  = dqn.test(env, nb_episodes=10, visualize=False)
rewards  = np.array(history.history['episode_reward'])
mean_reward = rewards.mean()
print(f"Media de recompensa obtenida en las pruebas: {mean_reward:.2f}")

✅  Pesos cargados desde /content/gdrive/My Drive/refuerzo/checkpoints/dqn_Model_1.h5
Testing for 10 episodes ...
Episode 1: reward: 17.000, steps: 927
Episode 2: reward: 17.000, steps: 809
Episode 3: reward: 8.000, steps: 648
Episode 4: reward: 6.000, steps: 512
Episode 5: reward: 7.000, steps: 631
Episode 6: reward: 13.000, steps: 927
Episode 7: reward: 11.000, steps: 654
Episode 8: reward: 5.000, steps: 415
Episode 9: reward: 8.000, steps: 644
Episode 10: reward: 15.000, steps: 655
Media de recompensa obtenida en las pruebas: 10.70


In [8]:
drive_root = "./"
WEIGHTS_DIR = os.path.join(drive_root, "weights")
CHECKPOINT_DIR = os.path.join(drive_root, "checkpoints")
# Asegurar que existen los directorios
os.makedirs(WEIGHTS_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print("Directorio de pesos:", WEIGHTS_DIR)
print("Directorio de checkpoints:", CHECKPOINT_DIR)
print("Archivos existentes en checkpoints:", os.listdir(CHECKPOINT_DIR))

Directorio de pesos: ./weights
Directorio de checkpoints: ./checkpoints
Archivos existentes en checkpoints: []


In [9]:
model_name2   = 'Model_2.1'
weights_path2 = os.path.join(CHECKPOINT_DIR, f'dqn_{model_name2}.h5')
steps_path2   = os.path.join(CHECKPOINT_DIR, f'dqn_{model_name2}_steps.txt')
memory_path2  = os.path.join(CHECKPOINT_DIR, f'memory_{model_name2}.pkl')
final_weights_path2 = os.path.join(WEIGHTS_DIR, f'dqn_{model_name2}.h5')

In [10]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model2 = Sequential()
model2.add(Permute((2, 3, 1), input_shape=input_shape))             # (84,84,4)
model2.add(Convolution2D(32, (8, 8), strides=(4, 4), activation='relu'))
model2.add(Convolution2D(64, (4, 4), strides=(2, 2), activation='relu'))
model2.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(Dense(nb_actions, activation='linear'))                  # logits
print(model2.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1606144   
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3

In [22]:
#Hiperparámetros
# Hiperparámetros optimizados para Space Invaders DQN
MEMORY_LIMIT = 100000          # Aumentado para mejor experiencia replay
NB_STEPS = 1000000             # Mínimo recomendado para resultados decentes
NB_STEPS_WARMUP = 50000        # Mantener, es adecuado
GAMMA = 0.99                   # Valor estándar para Atari games
TARGET_MODEL_UPDATE = 10000    # Mantener, es correcto
TRAIN_INTERVAL = 4             # Frecuencia de entrenamiento estándar para DQN
LEARNING_RATE = 0.00025        # Valor estándar y probado para DQN
FIT_NB_STEPS = 1000000         # Mantener igual que NB_STEPS
LOG_INTERVAL = 10000           # Aumentado para logs más espaciados
MODEL_CHECKPOINT_STEPS = 1000

In [23]:
#implementacion checkpoints
import tempfile


def atomic_pickle_dump(obj, path):
    """Escritura atómica: siempre obtienes pickle íntegro o nada."""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with tempfile.NamedTemporaryFile('wb', dir=os.path.dirname(path), delete=False) as tmp:
        pickle.dump(obj, tmp, protocol=pickle.HIGHEST_PROTOCOL)
        tmp.flush(); os.fsync(tmp.fileno())
    os.replace(tmp.name, path)          # rename atómico

def load_pickle_safe(path, default):
    """Devuelve pickle o `default` si falta/está vacío/corrupto."""
    if os.path.exists(path) and os.path.getsize(path) > 0:
        try:
            with open(path, 'rb') as f:
                return pickle.load(f)
        except Exception as e:           # EOFError, UnpicklingError...
            print(f"⚠️  Pickle dañado ({e}), usando objeto nuevo.")
    else:
        print("🆕  No había pickle previo.")
    return default

def load_steps(steps_file):
    if os.path.exists(steps_file):
        with open(steps_file, 'r') as f:
            try:
                data=int(f.read().strip())
                print(data)
                return data
            except ValueError:
                pass
    return 0

class SaveWeightsAndMemory(Callback):
    def __init__(self, w_path, s_path, m_path, interval_steps=10_000):
        super().__init__()
        self.w_path, self.s_path, self.m_path = w_path, s_path, m_path
        self.interval = interval_steps
        self.global_step = load_steps(s_path)
    def _save(self):
        self.model.save_weights(self.w_path, overwrite=True)
        atomic_pickle_dump(self.model.memory, self.m_path)
        open(self.s_path,'w').write(str(self.global_step))
        print(f'💾  checkpoint @ {self.global_step:,} pasos')
    def on_step_end(self, step,logs=None):
        self.global_step += 1
        if self.global_step % self.interval == 0:
            self._save()

In [24]:
steps_done2 = load_steps(steps_path2)          # función que ya tenías
remaining_steps2 = max(1, NB_STEPS - steps_done2)
resume_training = 0 < steps_done2 < NB_STEPS
print(steps_done2)
print(remaining_steps2)

57000
57000
943000


In [25]:
eps_policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),
                                  attr='eps',
                                  value_max=1, value_min=0.1, value_test=0.0,
                                  nb_steps=remaining_steps2)

In [26]:
if resume_training and os.path.exists(memory_path2):        # ➋ NUEVO
    print(f'✅ memoria restaurada ({memory_path2})')
    with open(memory_path2, 'rb') as f:
        memory2 = pickle.load(f)
else:
    print('🆕 memoria nueva')
    memory2 = SequentialMemory(limit=MEMORY_LIMIT,
                              window_length=WINDOW_LENGTH)
processor = AtariProcessor()

✅ memoria restaurada (./checkpoints\memory_Model_2.1.pkl)


In [27]:
dqn2 = DQNAgent(model=model2,
                nb_actions=nb_actions,
                memory=memory2,
                processor=processor,
                nb_steps_warmup=0 if resume_training else NB_STEPS_WARMUP,
                gamma=GAMMA,
                target_model_update=TARGET_MODEL_UPDATE,
                train_interval=TRAIN_INTERVAL,
                policy=eps_policy,
                enable_double_dqn=True,
                enable_dueling_network=True,
                dueling_type='avg')

dqn2.compile(Adam(learning_rate=LEARNING_RATE), metrics=['mae'])

In [28]:
if os.path.exists(weights_path2):
    dqn2.load_weights(weights_path2)
    print(f'✅  Pesos Model_2 cargados | pasos previos: {steps_done2:,}')
else:
    print('🆕  Model_2 inicia desde cero')

✅  Pesos Model_2 cargados | pasos previos: 57,000


In [29]:
save_cb2 = SaveWeightsAndMemory(weights_path2, steps_path2,memory_path2,interval_steps=MODEL_CHECKPOINT_STEPS)
try:
    dqn2.fit(env,
             nb_steps=remaining_steps2,
             callbacks=[save_cb2],
             log_interval=LOG_INTERVAL,
             verbose=2)
except KeyboardInterrupt:
    print("⏹️  Entrenamiento interrumpido manualmente")

final_steps2 = load_steps(steps_path2)
dqn2.save_weights(final_weights_path2, overwrite=True)
with open(steps_path2, 'w') as f:
    f.write(str(final_steps2))
print(f'🏁  Model_2 entrenado | pasos totales: {final_steps2:,}')

def _final_save(*_):
    atomic_pickle_dump(memory2, memory_path2)
    dqn2.save_weights(weights_path2, overwrite=True)
    open(steps_path2,'w').write(str(load_steps(steps_path2)))

57000
Training for 943000 steps ...


c:\ProgramData\Anaconda3\envs\refu\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    621/943000: episode: 1, duration: 16.216s, episode steps: 621, steps per second:  38, episode reward:  8.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.461 [0.000, 5.000],  loss: 0.005605, mae: 0.033875, mean_q: 0.044497, mean_eps: 0.999702
💾  checkpoint @ 58,000 pasos
   1147/943000: episode: 2, duration: 14.647s, episode steps: 526, steps per second:  36, episode reward:  5.000, mean reward:  0.010 [ 0.000,  1.000], mean action: 2.466 [0.000, 5.000],  loss: 0.007672, mae: 0.040282, mean_q: 0.052395, mean_eps: 0.999156
   1928/943000: episode: 3, duration: 19.704s, episode steps: 781, steps per second:  40, episode reward: 11.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.489 [0.000, 5.000],  loss: 0.005749, mae: 0.034799, mean_q: 0.045173, mean_eps: 0.998534
💾  checkpoint @ 59,000 pasos
   2388/943000: episode: 4, duration: 14.940s, episode steps: 460, steps per second:  31, episode reward:  6.000, mean reward:  0.013 [ 0.000,  1.000], mean action: 2.480 [0.

In [45]:
weights_filename2 = os.path.join(CHECKPOINT_DIR, f'dqn_{model_name2}.h5')

if not os.path.exists(weights_filename2):
    raise FileNotFoundError(f'❌  No se encontró {weights_filename2}')
dqn2.load_weights(weights_filename2)
print(f'✅  Pesos cargados desde {weights_filename2}')
history2  = dqn2.test(env, nb_episodes=10, visualize=False)
rewards2  = np.array(history2.history['episode_reward'])
mean_reward2 = rewards2.mean()
print(f"Media de recompensa obtenida en las pruebas: {mean_reward2:.2f}")

✅  Pesos cargados desde ./checkpoints\dqn_Model_2.1.h5
Testing for 10 episodes ...
Episode 1: reward: 12.000, steps: 807
Episode 2: reward: 30.000, steps: 1160
Episode 3: reward: 14.000, steps: 696
Episode 4: reward: 26.000, steps: 988
Episode 5: reward: 16.000, steps: 622
Episode 6: reward: 13.000, steps: 548
Episode 7: reward: 25.000, steps: 932
Episode 8: reward: 29.000, steps: 1401
Episode 9: reward: 15.000, steps: 657
Episode 10: reward: 21.000, steps: 844
Media de recompensa obtenida en las pruebas: 20.10


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---